# 問題例の作成

In [321]:
class TW():
    def __init__(self, e, l):
        self.e = e
        self.l = l

In [322]:
class Coordinate():
    def __init__(self, x, y):
        self.x = x
        self.y = y

In [323]:
class Customer():
    def __init__(self, d, tw, coo, penalty):
        self.demand = d
        self.tw = TW(tw[0], tw[1])
        self.coordinate = Coordinate(coo[0], coo[1])
        self.penalty = penalty

In [324]:
class Vehicle():
    def __init__(self, c, tour):
        self.capacity = c
        #self.loading = 0
        self.tour = tour

In [325]:
def ex():
    """
    V : 客集合
        N : 客数
        d : 需要
        tw : 時間枠
        (x, y) : 座標
        penalty : 時間枠を違反した場合にかかる、単位時間あたりの違反料
    K : 車両集合
        c : 容料
        #loading : 現在積んでいる荷物の量
        tour : 巡回路
    """
    import random
    random.seed(0)
    N = random.randint(10, 50)
    max_d = 10
    max_tw = 10
    max_co = 50
    V = {}
    sum_d = 0
    for i in range(N):
        d = random.uniform(-max_d, max_d)
        tw = sorted([random.uniform(0, max_tw) for i in range(2)])
        x, y = random.uniform(-max_co, max_co), random.uniform(-max_co, max_co)
        penalty = random.uniform(10, 100)
        V[i] = Customer(d, tw, (x, y), penalty)
        sum_d += d
        
    K = {}
    min_cap = max_d/7
    max_cap = max_d/5
    sum_cap = 0
    i = 0
    while sum_cap<sum_d:
        c = random.uniform(min_cap, max_cap)
        K[i] = Vehicle(c, {})
        sum_cap += c
        i += 1
    return V, K

In [326]:
def ex_small():
    """
    V : 客集合
        N : 客数
        d : 需要
        tw : 時間枠
        (x, y) : 座標
        penalty : 時間枠を違反した場合にかかる、単位時間あたりの違反料
    K : 車両集合
        c : 容料
        tour : 巡回路
    """
    import random
    random.seed(0)
    N = 5
    V = {}
    d_list = [1,-2,3,-4,5]
    tw_list = [(0,10),(1,6),(2,8),(3,7),(4,7)]
    coo_list = [(0,0) for i in range(5)]#[(0,2),(1,3),(2,6),(3,3),(4,10)]
    penalty_list = [10,10,10,20,20]
    for i in range(N):
        d = d_list[i]
        tw = tw_list[i]
        x, y = coo_list[i]
        penalty = penalty_list[i]
        V[i] = Customer(d, tw, (x, y), penalty)
        
    M = 3
    c_list = [2,4,6]
    K = {k:Vehicle(c_list[k], {}) for k in range(M)}
    K[0].tour = {0}
    K[1].tour = {1,2}
    K[2].tour = {3,4}
    
    return V, K

In [327]:
V, K = ex()

In [328]:
for i in V:
    print(i, end="\t")
    for condition in V[i].__dict__.values():
        print(condition, end="\n\t")
    print()

0	5.159088058806049
	80.54187301312953
	
1	-3.933745478421451
	35.36540599597335
	
2	5.116084083144479
	82.91955123969306
	
3	8.043319008791656
	52.4928443907442
	
4	-7.985975838632684
	52.93087988974453
	
5	7.3061985554328
	74.77342177635587
	
6	-2.023529155514625
	88.08424979435028
	
7	-5.121782462257361
	31.475433575369816
	
8	9.350805005802869
	55.71465782685165
	
9	8.656676484538135
	83.30201769622023
	
10	0.8056721394064787
	63.665817542479566
	
11	-2.3019770805467914
	65.1495861881746
	
12	3.1331877797925767
	93.10429143516525
	
13	6.849204462803648
	73.47550598689656
	
14	-4.487317573757457
	95.47883859089085
	
15	1.593900214912118
	81.39925757172017
	
16	-8.352540236067052
	31.87320598556706
	
17	4.629784415816955
	83.43217868802935
	
18	-7.987849595678076
	91.90144132291357
	
19	0.6839593652144789
	61.83576532283867
	
20	-2.1758118135434623
	96.49281522156501
	
21	-6.300561172051234
	12.050431810179255
	
22	-1.4876233606636564
	41.52645706568791
	
23	-6.393641969406243
	27.94

In [329]:
sum(i.demand for i in V.values())

-0.7368985565923367

In [330]:
sum(i.capacity for i in K.values())

0

# 問題を解く

## 巡回路を生成  
とりあえず、顧客の名前のインデックスが小さい順

## 巡回路を評価（各顧客の出発時刻を決める）  
どのくらい時間がかかるか調べる

In [331]:
# 顧客iへ車両が時刻arr_tに到着する場合の、顧客iの時間枠に関する違反料を計算する関数
def penalty(i, arr_t, V):
    #print("arr_t \t", arr_t)
    e_diff = arr_t - V[i].tw.e
    #print("V[i].tw.e \t", V[i].tw.e)
    #print("e_diff \t", e_diff)
    l_diff = arr_t - V[i].tw.l
    #print("V[i].tw.l \t", V[i].tw.l)
    #print("l_diff \t", l_diff)
    #print("e_diff*l_diff \t", e_diff*l_diff)
    #print("penalty per minute \t", V[i].penalty)
    return (-1*(e_diff<0)*e_diff + (l_diff>0)*l_diff) *V[i].penalty
    """
    if e_diff*l_diff > 0:
        return (-1*(e_diff<0)*e_diff + (l_diff>0)*l_diff) *V[i].penalty
    return 0
    """

In [332]:
def distance(i, j, V):
    x0 = V[i].coordinate.x
    y0 = V[i].coordinate.y
    x1 = V[j].coordinate.x
    y1 = V[j].coordinate.y
    return ((x0-x1)**2 + (y0-y1)**2)**(1/2)

In [337]:
import gurobipy as gp

# インスタンスの生成
m = gp.Model("LP_for_VRP_small")

# 定数を設定
V, K = ex_small()

# 変数を設定
"""
x_ik : 顧客iへ車両kが到着する時刻を表す変数
#z_ik : 車両kが顧客iに到着する時刻が、顧客iの時間枠に対して、どの程度違反しているかを表す変数
z_i : 車両が顧客iに到着する時刻が、顧客iの時間枠に対して、どの程度違反しているかを表す変数
"""
x = {(i,k):m.addVar(vtype=gp.GRB.CONTINUOUS, name=f"x({i},{k})") for k in K for i in K[k].tour}
#z = {(i,k):m.addVar(vtype=gp.GRB.CONTINUOUS, name=f"z({i},{k})") for k in K for i in K[k].tour}
z = {i:m.addVar(vtype=gp.GRB.CONTINUOUS, name=f"z({i})") for i in V}

m.update()

In [334]:
# 目的関数を設定
## 各顧客iの到着時刻x_ikと顧客iの時間枠を元に計算した違反料の和を最小化する
#m.setObjective(gp.quicksum(V[i].penalty*z[i,k] for k in K for i in K[k].tour), sense = gp.GRB.MINIMIZE)
m.setObjective(gp.quicksum(V[i].penalty*z[i] for i in V), sense = gp.GRB.MINIMIZE)

In [335]:
# 制約条件を設定
## 容料制約
"""for k in K:
    m.addConstr(gp.quicksum(V[i].demand for i in K[k].tour) <= K[k].capacity)"""
## 時間枠制約
for k in K:
    for i in K[k].tour:
        #m.addConstr(z[i,k] == penalty(i, x[i,k], V))
        m.addConstr(V[i].tw.e-z[i] <= x[i,k])
        m.addConstr(x[i,k] <= V[i].tw.l+z[i])
## 任意の車両の巡回路内における、各顧客への到着時刻の先行、後行に関する制約
for k in K:
    for i_p in K[k].tour:
        for i_f in K[k].tour:
            if i_p != i_f:
                m.addConstr(x[i_p, k] + distance(i_p, i_f, V) <= x[i_f, k])

In [336]:
# 最適化
m.optimize()
                
# 解の表示
if m.Status == gp.GRB.OPTIMAL:
    for k in K:
        for i in K[k].tour:
            print(f"車両{k}が顧客{i}に到着する時刻は、{x[i, k]}")
    print("最適値 : ", m.ObjVal)

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 14 rows, 10 columns and 28 nonzeros
Model fingerprint: 0xe952e9b9
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 6 rows and 4 columns
Presolve time: 0.03s
Presolved: 8 rows, 6 columns, 16 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.000000e+01   0.000000e+00      0s
       2    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.05 seconds
Optimal objective  0.000000000e+00
車両0が顧客0に到着する時刻は、<gurobi.Var x(0,0) (value 10.0)>
車両1が顧客1に到着する時刻は、<gurobi.Var x(1,1) (value 2.0)>
車両1が顧客2に到着する時刻は、<gurobi.Var x(2,1) (value 2.0)>
車両2が顧客3に到着する時刻は、<gurobi.Var x(3,2) (value 4.0)>
車両2が顧客4に到着する時刻は、<gurobi.Var x(4,2) (value 4.0)>
最適値 :  0.0
